# Sampling and Characterization Peptides Protocol

First create the peptide structure and then run it in *sander* or *pmemd.cuda*. This can be done easily with the tleap program placing this information in a file *tleap.in*:

In [16]:
tleap_in = """source leaprc.ff14SB
peptide = sequence { NASP ARG VAL HIS CPRO }
addions peptide Na+ 0
addions peptide Cl- 0
solvateBox peptide TIP3PBOX 13
saveAmberParm peptide p.1.top p.1.crd
quit
"""

open('tleap.in', 'w').write(tleap_in)

184

In [17]:
# Download input files
!gsutil cp gs://sbglabdata/src/gpu/leaprc.ff14SB .
!gsutil cp gs://sbglabdata/src/gpu/min.1.in .
!gsutil cp gs://sbglabdata/src/gpu/heat.*.inp .
!gsutil cp gs://sbglabdata/src/gpu/dmd.1.inp .
!gsutil cp gs://sbglabdata/src/gpu/eq.1.inp .    

Copying gs://sbglabdata/src/gpu/leaprc.ff14SB...
/ [1 files][  5.2 KiB/  5.2 KiB]                                                
Operation completed over 1 objects/5.2 KiB.                                      
Copying gs://sbglabdata/src/gpu/min.1.in...
/ [1 files][   74.0 B/   74.0 B]                                                
Operation completed over 1 objects/74.0 B.                                       
Copying gs://sbglabdata/src/gpu/heat.1.inp...
Copying gs://sbglabdata/src/gpu/heat.2.inp...                                   
Copying gs://sbglabdata/src/gpu/heat.3.inp...                                   
/ [3 files][  831.0 B/  831.0 B]                                                
Operation completed over 3 objects/831.0 B.                                      
Copying gs://sbglabdata/src/gpu/dmd.1.inp...
/ [1 files][  199.0 B/  199.0 B]                                                
Operation completed over 1 objects/199.0 B.                                      
Co

In [18]:
print(open('leaprc.ff14SB').read())

logFile leap.log
#
# ----- leaprc for loading the ff14SB force field
# ----- NOTE: this is designed for PDB format 3!
#    Uses frcmod.ff14SB for proteins; ff99bsc0 for DNA; ff99bsc0_chiOL3 for RNA
#
#	load atom type hybridizations
#
addAtomTypes {
	{ "H"   "H" "sp3" }
	{ "HO"  "H" "sp3" }
	{ "HS"  "H" "sp3" }
	{ "H1"  "H" "sp3" }
	{ "H2"  "H" "sp3" }
	{ "H3"  "H" "sp3" }
	{ "H4"  "H" "sp3" }
	{ "H5"  "H" "sp3" }
	{ "HW"  "H" "sp3" }
	{ "HC"  "H" "sp3" }
	{ "HA"  "H" "sp3" }
	{ "HP"  "H" "sp3" }
	{ "HZ"  "H" "sp3" }
	{ "OH"  "O" "sp3" }
	{ "OS"  "O" "sp3" }
	{ "O"   "O" "sp2" }
	{ "O2"  "O" "sp2" }
	{ "OP"  "O" "sp2" }
	{ "OW"  "O" "sp3" }
	{ "CT"  "C" "sp3" }
	{ "CX"  "C" "sp3" }
	{ "C8"  "C" "sp3" }
	{ "2C"  "C" "sp3" }
	{ "3C"  "C" "sp3" }
	{ "CH"  "C" "sp3" }
	{ "CS"  "C" "sp2" }
	{ "C"   "C" "sp2" }
	{ "CO"   "C" "sp2" }
	{ "C*"  "C" "sp2" }
	{ "CA"  "C" "sp2" }
	{ "CB"  "C" "sp2" }
	{ "CC"  "C" "sp2" }
	{ "CN"  "C" "sp2" }
	{ "CM"  "C" "sp2" }
	{ "CK"  "C" "sp2" }
	{ "CQ"  "C" "s

## Step 1

Each residue sequence has the three letter code for each amino acid, althought the first will be have N and the last C.

tleap is used to generate the model. This will produce 2 files. A topology one ((p.1.top) and coordinates (p.1.crd). From now on we will use these two files to run the dynamics. Topology file will always be the same while the coordinate file will change in every step

Run the program like this:

In [19]:
!~/amber20/bin/tleap -s -f tleap.in

-I: Adding /home/sebastian/amber20/dat/leap/prep to search path.
-I: Adding /home/sebastian/amber20/dat/leap/lib to search path.
-I: Adding /home/sebastian/amber20/dat/leap/parm to search path.
-I: Adding /home/sebastian/amber20/dat/leap/cmd to search path.
-s: Ignoring startup file: leaprc
-f: Source tleap.in.

Welcome to LEaP!
Sourcing: ./tleap.in
----- Source: ./leaprc.ff14SB
----- Source of ./leaprc.ff14SB done
Log file: ./leap.log
Loading parameters: /home/sebastian/amber20/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /home/sebastian/amber20/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /home/sebastian/amber20/dat/leap/lib/amino12.lib
Loading library: /home/sebastian/amber20/dat/leap/lib/aminoct12.lib
Loading library: /home/sebastian/amber20/dat/leap/lib/aminont12.lib
Loading library: /home/sebastia

## Step 2

**Minimize**: From here we will use *sander or pmemd.cuda* from Amber. To minimize we need 3 input files. Topology (p.1.top), coordinates (p.1.crd) and the one used to input variables, in this case is min.1.in, here is the content:

In [20]:
print(open('min.1.in').read())

Minimizacion
&cntrl
imin=1,
maxcyc=8000,
ncyc=7500,
cut=9,
ntpr=200,
&end



How to minimize:

In [21]:
!time ~/amber20/bin/pmemd.cuda -O -i min.1.in -o min.1.out -p p.1.top -c p.1.crd -r min.1.crd

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL

real	0m6.010s
user	0m5.563s
sys	0m0.404s


The input file is the one ended in .in, the output is the .out. The .top topology file generated with *tleap*, p.1.crd is the coordinates files generated by *tleap*. min.1.crd is the coordinate output file that will be used in the next step.

## Step 3

**Heating**: Use this input (heat.1.inp) so the system goes from 0 K to 100 K

In [22]:
print(open('heat.1.inp').read())

35 heating
&cntrl
imin=0,irest=0,ntx=1,ntb=1,cut=9.0,ntp=0,nmropt=1,
ntt=3,nstlim=10000,ntpr=500,ntwx=10000,dt=0.002,ioutfm=1,
ntwr=10000,gamma_ln=1.0,tol=0.0000001,ntf=2,ntc=2,
&end
&wt
type="TEMP0", istep1=0, istep2=10000, value1=0.0, value2=100.0,
&end
&wt
type="END",
&end



Run this with the next command:

In [23]:
!time ~/amber20/bin/pmemd.cuda -O -i heat.1.inp -o heat.1.out -p p.1.top -c min.1.crd -r heat.1.crd -x heat.1.traj

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL

real	0m4.852s
user	0m4.374s
sys	0m0.440s


This run will output the heat.1.crd with positions and speeds related to the system at 100 K.
Use this input (heat.2.inp) to move the system from 100 K to 200 K

In [24]:
print(open('heat.2.inp').read())

35 heating
&cntrl
imin=0,irest=1,ntx=5,ntb=2,cut=9.0,ntp=2,nmropt=1,
ntt=3,nstlim=5000,ntpr=500,ntwx=5000,dt=0.002,ioutfm=1,
ntwr=5000,gamma_ln=1.0,tol=0.0000001,ntf=2,ntc=2,
&end
&wt
type="TEMP0", istep1=0, istep2=5000, value1=100.0, value2=200.0,
&end
&wt
type="END",
&end



Run the following:

In [25]:
!time ~/amber20/bin/pmemd.cuda -O -i heat.2.inp -o heat.2.out -p p.1.top -c heat.1.crd -r heat.2.crd -x heat.2.traj

ERROR: Calculation halted.  Periodic box dimensions have changed too much from their initial values.
  Your system density has likely changed by a large amount, probably from
  starting the simulation from a structure a long way from equilibrium.

  [Although this error can also occur if the simulation has blown up for some reason]

  The GPU code does not automatically reorganize grid cells and thus you
  will need to restart the calculation from the previous restart file.
  This will generate new grid cells and allow the calculation to continue.
  It may be necessary to repeat this restarting multiple times if your system
  is a long way from an equilibrated density.

  Alternatively you can run with the CPU code until the density has converged
  and then switch back to the GPU code.


real	0m3.449s
user	0m3.081s
sys	0m0.341s
